<a href="https://colab.research.google.com/github/Reem-Aboutaleb/multimodal-stress-detection/blob/main/03_Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from scipy.signal import find_peaks

# Load the dataset (same as before)
df = pd.read_csv("sample_signals.csv")
ppg = df['ppg']
eda = df['eda']
label = df['label']

# --- PPG Feature Extraction ---
peaks, _ = find_peaks(ppg, distance=50)
ibi = np.diff(peaks)

def hrv_features(ibi):
    if len(ibi) < 2:
        return {'ibi_mean': 0, 'ibi_sdnn': 0, 'ibi_rmssd': 0}
    return {
        'ibi_mean': np.mean(ibi),
        'ibi_sdnn': np.std(ibi),
        'ibi_rmssd': np.sqrt(np.mean(np.square(np.diff(ibi))))
    }

hrv = hrv_features(ibi)

# --- EDA Feature Extraction ---
eda_mean = np.mean(eda)
eda_std = np.std(eda)
eda_max = np.max(eda)
eda_min = np.min(eda)

# --- Combine All Features ---
features = {
    'ppg_ibi_mean': hrv['ibi_mean'],
    'ppg_ibi_sdnn': hrv['ibi_sdnn'],
    'ppg_ibi_rmssd': hrv['ibi_rmssd'],
    'eda_mean': eda_mean,
    'eda_std': eda_std,
    'eda_max': eda_max,
    'eda_min': eda_min
}

features_df = pd.DataFrame([features])
features_df.to_csv("extracted_features.csv", index=False)
print("✅ Features saved to extracted_features.csv")
df = pd.read_csv("extracted_features.csv")
from scipy.signal import welch

# --- pNN50 ---
def pnn50(ibi):
    diff_ibi = np.abs(np.diff(ibi))
    return 100.0 * np.sum(diff_ibi > 50) / len(diff_ibi)

# --- Frequency Domain HRV ---
def frequency_domain_features(ibi, fs=4):
    if len(ibi) < 2:
        return {'HRV_LF': 0, 'HRV_HF': 0, 'LF_HF_ratio': 0}

    fxx, pxx = welch(ibi, fs=fs)
    lf_band = (fxx >= 0.04) & (fxx <= 0.15)
    hf_band = (fxx >= 0.15) & (fxx <= 0.4)

    lf = np.trapz(pxx[lf_band], fxx[lf_band])
    hf = np.trapz(pxx[hf_band], fxx[hf_band])
    return {
        'HRV_LF': lf,
        'HRV_HF': hf,
        'LF_HF_ratio': lf / hf if hf != 0 else 0
    }

# Calculate new HRV features
pnn_50 = pnn50(ibi)
freq_feats = frequency_domain_features(ibi)
from scipy.signal import find_peaks

# Find EDA peaks
eda_peaks, properties = find_peaks(eda, distance=50, prominence=0.02)
scr_count = len(eda_peaks)
scr_mean_amp = np.mean(properties["prominences"]) if scr_count > 0 else 0
features = {
    'ppg_ibi_mean': hrv['ibi_mean'],
    'ppg_ibi_sdnn': hrv['ibi_sdnn'],
    'ppg_ibi_rmssd': hrv['ibi_rmssd'],
    'pnn50': pnn_50,
    'HRV_LF': freq_feats['HRV_LF'],
    'HRV_HF': freq_feats['HRV_HF'],
    'LF_HF_ratio': freq_feats['LF_HF_ratio'],
    'eda_mean': eda_mean,
    'eda_std': eda_std,
    'eda_max': eda_max,
    'eda_min': eda_min,
    'eda_scr_count': scr_count,
    'eda_scr_mean_amp': scr_mean_amp
}
features_df = pd.DataFrame([features])
features_df.to_csv("extracted_features_advanced.csv", index=False)






✅ Features saved to extracted_features.csv


/usr/local/lib/python3.11/dist-packages/scipy/signal/_spectral_py.py:600: UserWarning: nperseg = 256 is greater than input length  = 90, using nperseg = 90
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,
<ipython-input-2-4be0ab3cd119>:63: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  lf = np.trapz(pxx[lf_band], fxx[lf_band])
<ipython-input-2-4be0ab3cd119>:64: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  hf = np.trapz(pxx[hf_band], fxx[hf_band])
